In [6]:
import json
import numpy as np
import string
import re
import h5py

In [7]:
with open('train-v1.1.json') as json_data:
    d = json.load(json_data)

In [8]:
dataset = d['data']

In [9]:
len(dataset)

442

In [10]:
dataset[0]

{u'paragraphs': [{u'context': u'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
   u'qas': [{u'answers': [{u'answer_start': 515,
       u'text': u'Saint Bernadette Soubirous'}],
     u'id': u'5733be284776f41900661182',
     u'question': u'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'},
    {u'answers': [{u'answer_start':

In [11]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


In [12]:
# stores word to index
word_dict = {}
#stores index to word
ind_dict = {}

index = 1

In [13]:
def modify(cont):
    global index
    temp_list = []
    temp_str = ""
    for i in range(len(cont)):
        if cont[i] == '"' or cont[i] == '/' or cont[i] == ';' or cont[i] == ',':
            continue
                        
        if cont[i] == '?' or cont[i] == ' ' or cont[i] == '.':
            if cont[i] == ',' or cont[i] == '.' or cont[i] == '?':
                i += 1
                            
            word = temp_str.lower()
            temp_str = ""
            temp_index = 0
                        
            if word not in word_dict:
                word_dict[word] = index
                ind_dict[index] = word
                temp_index = index
                index += 1
            else:
                temp_index = word_dict[word]
                            
            temp_list.append(temp_index)
        else:
            temp_str += cont[i]
    return temp_list
                

In [14]:
context_list = []
question_list = []
answer_list = []
answer_begin = []
answer_end = []
id_list = []

for article in dataset:
    for paragraph in article['paragraphs']:
        for qa in paragraph['qas']:
            id_list.append(qa['id'])
            for ans in qa['answers']:
                # append both context and questions many times for more than one question/answer
                ques = qa['question']
                if len(modify(ques)) < 50:
                    question_list.append(modify(ques))
                    cont = paragraph['context']
                    context_list.append(modify(cont))
                
                    an = ans['text']
                    answer_list.append(modify(an))
                
                    answer_begin.append(ans['answer_start'])
                    answer_end.append(ans['answer_start']+len(ans['text']))

In [34]:
print word_dict

{'': 8, u'architecturally': 1, u'basilica': 39, u'statues': 68, u'ad': 34, u'gold': 12, u'golden': 15, u'france': 51, u'is': 14, u'venite': 33, u'it': 26, u'connects': 65, u'(and': 61, u'bernadette': 56, u'dame': 79, u'replica': 48, u'through': 66, u'at': 49, u'in': 21, u'omnes': 36, u'appeared': 54, u'reputedly': 53, u'1858': 58, u'catholic': 6, u'what': 77, u'sits': 83, u'end': 59, u'and': 24, u'to': 38, u'soubirous': 57, u'top': 85, u'grotto': 43, u'character': 7, u'arms': 30, u'next': 37, u'beside': 80, u'virgin': 18, u'behind': 42, u'whom': 73, u'structure': 82, u'sacred': 40, u'dome)': 69, u'main': 10, u'copper': 27, u'mary': 19, u'stone': 72, u'3': 67, u'building': 23, u'modern': 71, u'saint': 55, u'allegedly': 75, u'that': 64, u'notre': 78, u'atop': 9, u'dome': 13, u'prayer': 46, u'direct': 62, u'me': 35, u'statue': 16, u'upraised': 31, u'front': 22, u'immediately': 20, u'line': 63, u'with': 29, u'legend': 32, u'appear': 76, u'christ': 28, u'a': 5, u'on': 84, u'school': 3, u'ha

In [35]:
print ind_dict

{1: u'architecturally', 2: u'the', 3: u'school', 4: u'has', 5: u'a', 6: u'catholic', 7: u'character', 8: '', 9: u'atop', 10: u'main', 11: u"building's", 12: u'gold', 13: u'dome', 14: u'is', 15: u'golden', 16: u'statue', 17: u'of', 18: u'virgin', 19: u'mary', 20: u'immediately', 21: u'in', 22: u'front', 23: u'building', 24: u'and', 25: u'facing', 26: u'it', 27: u'copper', 28: u'christ', 29: u'with', 30: u'arms', 31: u'upraised', 32: u'legend', 33: u'venite', 34: u'ad', 35: u'me', 36: u'omnes', 37: u'next', 38: u'to', 39: u'basilica', 40: u'sacred', 41: u'heart', 42: u'behind', 43: u'grotto', 44: u'marian', 45: u'place', 46: u'prayer', 47: u'reflection', 48: u'replica', 49: u'at', 50: u'lourdes', 51: u'france', 52: u'where', 53: u'reputedly', 54: u'appeared', 55: u'saint', 56: u'bernadette', 57: u'soubirous', 58: u'1858', 59: u'end', 60: u'drive', 61: u'(and', 62: u'direct', 63: u'line', 64: u'that', 65: u'connects', 66: u'through', 67: u'3', 68: u'statues', 69: u'dome)', 70: u'simple', 

In [37]:
print context_list
print
print question_list
print
print answer_list
print
print answer_begin
print
print answer_end

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14, 5, 15, 16, 17, 2, 18, 19, 8, 20, 21, 22, 17, 2, 10, 23, 24, 25, 26, 14, 5, 27, 16, 17, 28, 29, 30, 31, 29, 2, 32, 33, 34, 35, 36, 8, 37, 38, 2, 10, 23, 14, 2, 39, 17, 2, 40, 41, 8, 20, 42, 2, 39, 14, 2, 43, 5, 44, 45, 17, 46, 24, 47, 8, 26, 14, 5, 48, 17, 2, 43, 49, 50, 51, 52, 2, 18, 19, 53, 54, 38, 55, 56, 57, 21, 58, 8, 49, 2, 59, 17, 2, 10, 60, 61, 21, 5, 62, 63, 64, 65, 66, 67, 68, 24, 2, 12, 69, 14, 5, 70, 71, 72, 16, 17, 19], [1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11, 12, 13, 14, 5, 15, 16, 17, 2, 18, 19, 8, 20, 21, 22, 17, 2, 10, 23, 24, 25, 26, 14, 5, 27, 16, 17, 28, 29, 30, 31, 29, 2, 32, 33, 34, 35, 36, 8, 37, 38, 2, 10, 23, 14, 2, 39, 17, 2, 40, 41, 8, 20, 42, 2, 39, 14, 2, 43, 5, 44, 45, 17, 46, 24, 47, 8, 26, 14, 5, 48, 17, 2, 43, 49, 50, 51, 52, 2, 18, 19, 53, 54, 38, 55, 56, 57, 21, 58, 8, 49, 2, 59, 17, 2, 10, 60, 61, 21, 5, 62, 63, 64, 65, 66, 67, 68, 24, 2, 12, 69, 14, 5, 70, 71, 72, 16, 17, 19], [1, 2, 3, 4, 5, 6, 7, 8

In [38]:
for ind in context_list[1]:
    print ind_dict[ind],

architecturally the school has a catholic character  atop the main building's gold dome is a golden statue of the virgin mary  immediately in front of the main building and facing it is a copper statue of christ with arms upraised with the legend venite ad me omnes  next to the main building is the basilica of the sacred heart  immediately behind the basilica is the grotto a marian place of prayer and reflection  it is a replica of the grotto at lourdes france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858  at the end of the main drive (and in a direct line that connects through 3 statues and the gold dome) is a simple modern stone statue of mary


In [15]:
mx = 0
for i in range(len(context_list)):
    if len(context_list[i]) >= mx:
        print mx,
        mx = len(context_list[i])
print mx

0 130 130 130 130 130 226 226 226 226 226 232 232 232 232 232 260 260 260 260 260 266 266 266 266 266 293 293 293 293 293 297 297 297 297 297 307 307 307 307 307 333 333 333 333 333 333 333 333 333 333 333 333 333 333 338 338 338 338 338 338 338 338 338 338 338 338 338 368 368 368 368 519 519 519 519 554 554 554 554 554 694 694 694 694 694


In [16]:
context_array = np.zeros((len(context_list), 700), dtype=np.int)
question_array = np.zeros((len(question_list), 50), dtype=np.int)
answer_array = np.zeros((len(answer_list), 50), dtype=np.int)
begin_array = np.zeros((len(answer_begin), ), dtype=np.int)
end_array = np.zeros((len(answer_end), ), dtype=np.int)

for i in range(len(context_list)):
    for j in range(len(context_list[i])):
        context_array[i][j] = context_list[i][j]
        
for i in range(len(question_list)):
    for j in range(len(question_list[i])):
        question_array[i][j] = question_list[i][j]
        
        
for i in range(len(answer_list)):
    for j in range(len(answer_list[i])):
        answer_array[i][j] = answer_list[i][j]
            
for i in range(len(answer_begin)):
    begin_array[i] = answer_begin[i]
    
for i in range(len(answer_end)):
    end_array[i] = answer_end[i]

print context_array.shape
print question_array.shape
print answer_array.shape
print begin_array.shape
print end_array.shape

(87598, 700)
(87598, 50)
(87598, 50)
(87598,)
(87598,)


In [17]:
with h5py.File('context.h5', 'w') as hf:
    hf.create_dataset('context', data=context_array)
with h5py.File('questions.h5', 'w') as hf:
    hf.create_dataset('questions', data=question_array)
with h5py.File('answers.h5', 'w') as hf:
    hf.create_dataset('answers', data=answer_array)
with h5py.File('begin.h5', 'w') as hf:
    hf.create_dataset('begin', data=begin_array)
with h5py.File('end.h5', 'w') as hf:
    hf.create_dataset('end', data=end_array)


In [18]:
# saving dictionaries
np.save('word_to_indx.npy', word_dict)
np.save('indx_to_word', ind_dict)

In [19]:
word_dict = np.load('word_to_indx.npy').item()
ind_dict = np.load('indx_to_word.npy').item()

In [28]:
ind_dict[135]

u'literature'

In [21]:
word_dict['literature']

135

In [22]:
len(word_dict)

119689